In [86]:
import pickle
import re 
import pickle
import os
import json
import requests
import numpy as np
import pandas as pd
import scipy.stats as sts
from tqdm import tqdm
from datetime import timedelta, datetime
from scipy.special import expit as sigmoid
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import Ridge, LinearRegression, LogisticRegression

# 1. Прочитайте и проанализируйте данные
Прочитайте и проанализируйте данные, выберите турниры, в которых есть данные о составах команд и повопросных результатах (поле mask в results.pkl). Для унификации предлагаю:
- взять в тренировочный набор турниры с dateStart из 2019 года; 
- в тестовый — турниры с dateStart из 2020 года.

In [5]:
with open("data/tournaments.pkl", "rb") as f:
    tournaments_raw_data = pickle.load(f)

tournaments_df = pd.DataFrame(tournaments_raw_data)
tournaments_df = tournaments_df.transpose()
tournaments_df.dateStart = pd.to_datetime(tournaments_df.dateStart)
tournaments_df.head()

,id,name,dateStart,dateEnd,type,season,orgcommittee,synchData,questionQty
1,1,Чемпионат Южного Кавказа,2003-07-25 00:00:00+04:00,2003-07-27T00:00:00+04:00,"{'id': 2, 'name': 'Обычный'}",/seasons/1,[],None,None
2,2,Летние зори,2003-08-09 00:00:00+04:00,2003-08-09T00:00:00+04:00,"{'id': 2, 'name': 'Обычный'}",/seasons/1,[],None,None
3,3,Турнир в Ижевске,2003-11-22 00:00:00+03:00,2003-11-24T00:00:00+03:00,"{'id': 2, 'name': 'Обычный'}",/seasons/2,[],None,None
4,4,Чемпионат Украины. Переходной этап,2003-10-11 00:00:00+04:00,2003-10-12T00:00:00+04:00,"{'id': 2, 'name': 'Обычный'}",/seasons/2,[],None,None
5,5,Бостонское чаепитие,2003-10-10 00:00:00+04:00,2003-10-13T00:00:00+04:00,"{'id': 2, 'name': 'Обычный'}",/seasons/2,[],None,None


In [6]:
with open('data/players.pkl', 'rb') as file:
    players_raw_data = pickle.load(file)
    
players_df = pd.DataFrame(players_raw_data)
players_df = players_df.transpose()
players_df.head()

,id,name,patronymic,surname
1,1,Алексей,None,Абабилов
10,10,Игорь,,Абалов
11,11,Наталья,Юрьевна,Абалымова
12,12,Артур,Евгеньевич,Абальян
13,13,Эрик,Евгеньевич,Абальян


In [7]:
with open("data/results.pkl", "rb") as f:
    results_raw_data = pickle.load(f)

for k in list(results_raw_data.keys())[:5]:
    print("\n" + "-" * 20)
    print("{}:".format(k))
    for row in results_raw_data[k][:5]:
        print(row)


--------------------
1:
{'team': {'id': 242, 'name': 'Команда Азимова', 'town': {'id': 21, 'name': 'Баку'}}, 'mask': None, 'current': {'name': 'Команда Азимова', 'town': {'id': 21, 'name': 'Баку'}}, 'questionsTotal': 0, 'synchRequest': None, 'position': 1, 'controversials': [], 'flags': [], 'teamMembers': [{'flag': None, 'usedRating': 0, 'rating': 0, 'player': {'id': 476, 'name': 'Анар', 'patronymic': 'Беюкага оглы', 'surname': 'Азимов'}}, {'flag': None, 'usedRating': 0, 'rating': 0, 'player': {'id': 878, 'name': 'Фариз', 'patronymic': 'Наим оглы', 'surname': 'Аликишибеков'}}, {'flag': None, 'usedRating': 0, 'rating': 0, 'player': {'id': 1872, 'name': 'Аднан', 'patronymic': 'Фариз оглы', 'surname': 'Ахундов'}}, {'flag': None, 'usedRating': 0, 'rating': 0, 'player': {'id': 13721, 'name': 'Балаш', 'patronymic': 'Алекпер оглы', 'surname': 'Касумов'}}, {'flag': None, 'usedRating': 0, 'rating': 0, 'player': {'id': 22132, 'name': 'Анар', 'patronymic': 'Джафар оглы', 'surname': 'Наджафли'}},

In [127]:
TRAIN_YEAR = 2019
TEST_YEAR = 2020

TRAIN_FILE_PATH = "preapred_train.csv"
TEST_FILE_PATH = "preapred_test.csv"
RELOAD = False

question_to_tournament = {}

def create_preprocessed_dfs(train_file, test_file):
    
    cnt = 0
    cur_question_id = 0

    for key in tqdm(tournaments_raw_data.keys(), position=0, leave=False):
        tournaments_row = tournaments_raw_data[key]
        results_row = results_raw_data[key]
        
        tournament_year = datetime.fromisoformat(tournaments_row['dateStart']).replace(tzinfo=None).year
        # filtering values not in train or validation set

        if not(tournament_year == TRAIN_YEAR or tournament_year == TEST_YEAR):
            continue

        # filtering tournaments without results
        if not results_row:
            continue
                       
        questions_num = sum(tournaments_row['questionQty'].values())
        
        questions_added = False
        # processing teams
        for team in results_row:
            answers_mask = team.get("mask")
            # filtering tournaments wiht different questions number 
            if answers_mask is None or len(answers_mask) != questions_num:
                continue
                
            # filtering unknown status of answers
            answers_mask = re.sub(r"X|\?", "0", answers_mask)
            questions_added = True            
            team_id = team["team"]["id"]
            
            for i, res in enumerate(answers_mask):
                question_to_tournament[i + cur_question_id] = key
                for member in team["teamMembers"]:
                    new_line = [
                        member["player"]["id"], # player_id
                        team_id,                # team_id
                        i + cur_question_id,    # question_id
                        int(res)                # is_correct
                    ]
                    new_line = "\t".join([str(r) for r in new_line]) + "\n"
                    
                    if tournament_year == TRAIN_YEAR:
                        train_file.write(new_line)
                    else:
                        test_file.write(new_line)
        if questions_added:
            cur_question_id += questions_num

In [128]:
RELOAD = False

if not(os.path.exists(TRAIN_FILE_PATH) and os.path.exists(TEST_FILE_PATH)) or RELOAD:
    with open(TRAIN_FILE_PATH, "w+") as train_file, open(TEST_FILE_PATH, "w+") as test_file:
        create_preprocessed_dfs(train_file, test_file)

train_df = pd.read_csv(TRAIN_FILE_PATH, sep="\t", names=["player_id", "team_id", "question_id", "is_correct"])
test_df = pd.read_csv(TEST_FILE_PATH, sep="\t", names=["player_id", "team_id", "question_id", "is_correct"])

train_df.head()

,player_id,team_id,question_id,is_correct
0,6212,45556,0,1
1,18332,45556,0,1
2,18036,45556,0,1
3,22799,45556,0,1
4,15456,45556,0,1


In [10]:
test_tournament_ids = [
    key
    for key in tournaments_raw_data.keys()
    if datetime.fromisoformat(tournaments_raw_data[key]['dateStart']).replace(tzinfo=None).year == TEST_YEAR
]

In [11]:
encoder = OneHotEncoder(handle_unknown='ignore')

X_train = encoder.fit_transform(train_df[['player_id', 'question_id']])
X_test = encoder.transform(test_df[['player_id', 'question_id']])

y_train = train_df['is_correct']
y_test = test_df['is_correct']

И еще данные для статистики:

In [12]:
player_questions_stat = {}

for key in tqdm(tournaments_raw_data.keys(), position=0, leave=False):
    tournaments_row = tournaments_raw_data[key]
    results_row = results_raw_data[key]
    
    tournament_year = datetime.fromisoformat(tournaments_row['dateStart']).replace(tzinfo=None).year
    # filtering values not in train or validation set

    if not(tournament_year == TRAIN_YEAR or tournament_year == TEST_YEAR):
        continue

    # filtering tournaments without results
    if not results_row:
        continue

    questions_num = sum(tournaments_row['questionQty'].values())
    
    # processing teams
    for team in results_row:
        answers_mask = team.get("mask")
        if answers_mask is None or len(answers_mask) != questions_num:
            continue

        # filtering unknown status of answers
        answers_mask = re.sub(r"X|\?", '0', answers_mask)
        
        team_id = team["team"]["id"]
        for member in team["teamMembers"]:
            player_id = member['player']['id']
            player_stat = player_questions_stat.get(player_id)
            if player_stat is None:
                player_stat = {
                    "total_questions": 0,
                    "answered_questions": 0,
                    "total_games": 0
                }
            player_stat["total_games"] += 1
            player_stat["total_questions"] += len(answers_mask)
            player_stat["answered_questions"] += len(answers_mask.replace("0", ""))
            
            player_questions_stat[player_id] = player_stat

In [129]:
tournament_to_question = {}

for question_id, tournament_id in question_to_tournament.items():
    questions_arr = tournament_to_question.get(tournament_id, [])
    questions_arr.append(question_id)
    tournament_to_question[tournament_id] = questions_arr    

# 2. baseline-модель
Постройте baseline-модель на основе линейной или логистической регрессии, которая будет обучать рейтинг-лист игроков. Замечания и подсказки:
- повопросные результаты — это фактически результаты броска монетки, и их предсказание скорее всего имеет отношение к бинарной классификации;
- в разных турнирах вопросы совсем разного уровня сложности, поэтому модель должна это учитывать; скорее всего, модель должна будет явно обучать не только силу каждого игрока, но и сложность каждого вопроса;
- для baseline-модели можно забыть о командах и считать, что повопросные результаты команды просто относятся к каждому из её игроков.


In [14]:
train_df.nunique()

player_id      57429
team_id        11420
question_id    33381
is_correct         2
dtype: int64

In [15]:
logreg = LogisticRegression(penalty='l2', solver='liblinear').fit(X_train, y_train)

In [16]:
num_uniq_cols = train_df.nunique()
num_players = num_uniq_cols[0]
num_questions = num_uniq_cols[2]

player_weights = logreg.coef_[0][:num_players]
question_wights = logreg.coef_[0][num_players:]

top_100_data = []
top_100_players = []

for i, coef_idx in enumerate(np.argsort(player_weights)):
    
    player_id = encoder.categories_[0][coef_idx]
    player = players_raw_data[player_id]
    player_fio = "{} {} {}".format(player["surname"], player["name"], player["patronymic"]).strip()
    player_stat = player_questions_stat[player_id]

    if player_stat["total_questions"] < 1000:
        continue
        
    top_100_data.append({
        "player_id": player_id,
        "name": player_fio,
        "total_games": player_stat["total_games"],
        "total_questions": player_stat["total_questions"],
        "answered_questions": player_stat["answered_questions"],
        "answered/total":  player_stat["answered_questions"] / player_stat["total_questions"],
        "weight": player_weights[coef_idx],
    })
    top_100_players.append(player_fio)
      
sorted_rating = sorted(top_100_data, key=lambda x: x["weight"], reverse=True)
rating_df = pd.DataFrame(sorted_rating)

rating_df.head(50)

,player_id,name,total_games,total_questions,answered_questions,answered/total,weight
0,27403,Руссо Максим Михайлович,66,2624,2075,0.790777,4.159502
1,4270,Брутер Александра Владимировна,76,3056,2433,0.796139,4.032029
2,28751,Семушин Иван Николаевич,109,4342,3402,0.783510,3.987432
3,27822,Савченков Михаил Владимирович,94,3820,2955,0.773560,3.897539
4,30270,Спешков Сергей Леонидович,110,4669,3518,0.753480,3.819819
5,30152,Сорожкин Артём Сергеевич,164,6453,4874,0.755308,3.818941
6,20691,Мереминский Станислав Григорьевич,46,2024,1461,0.721838,3.701834
7,18036,Левандовский Михаил Ильич,39,1734,1261,0.727220,3.645077
8,26089,Прокофьева Ирина Сергеевна,32,1321,953,0.721423,3.600501
9,22799,Николенко Сергей Игоревич,59,2619,1766,0.674303,3.585115


С сайта ["Официальный рейтинг сообщества знатоков"](https://rating.chgk.info/players.php) был скопирован топ-100 игроков для сравнения с теми результатами, что были получены.

In [17]:
with open('rating.chgk.100.json') as json_file:
    real_top_100_players = json.load(json_file)

In [18]:
len(set(top_100_players) & set(real_top_100_players))

86

Порядок таких игроков не очень совпадает, но сами списки сами игроков, попавших в топ-100 довольно близки.

# 3. Качество рейтинг-системы
Качество рейтинг-системы оценивается качеством предсказаний результатов турниров. Но сами повопросные результаты наши модели предсказывать вряд ли смогут, ведь неизвестно, насколько сложными окажутся вопросы в будущих турнирах; да и не нужны эти предсказания сами по себе. Поэтому:
- предложите способ предсказать результаты нового турнира с известными составами, но неизвестными вопросами, в виде ранжирования команд;
- в качестве метрики качества на тестовом наборе давайте считать ранговые корреляции Спирмена и Кендалла (их можно взять в пакете scipy) между реальным ранжированием в результатах турнира и предсказанным моделью, усреднённые по тестовому множеству турниров.

In [19]:
def avg_weights(arr):
    if arr:
        return sum(arr) / len(arr)
    return 0

In [68]:
def get_pid_to_weight(model, encoder):
    return {
        encoder.categories_[0][coef_idx]: weight
        for coef_idx, weight in enumerate(model.coef_[0][:num_players])
    }

In [71]:
def get_predictions(tournament_ids, results_data, pid_to_weight, aggregate_weight=avg_weights):
    pred_results = {}
    for tour_id in tournament_ids:
        tournament = results_data.get(tour_id)
        if not tournament:
            continue 
        predicted_tournament_results = []
        for team in tournament:
            # get weight
            weights = [pid_to_weight.get(member["player"]["id"], 0) for member in team["teamMembers"]]
            team_weight = aggregate_weight(weights)
            # get prediction for team
            predicted_tournament_results.append([team["team"]["id"], team_weight])
        pred_results[tour_id] = np.array(predicted_tournament_results, dtype=int)
        
    return pred_results

def get_predictions_old(tournament_ids, results_data, member_to_idx, aggregate_weight=avg_weights):
    pred_results = {}
    for tour_id in tournament_ids:
        tournament = results_data.get(tour_id)
        if not tournament:
            continue 
        predicted_tournament_results = []
        for team in tournament:
    
            member_idxs = [member_to_idx.get(member["player"]["id"], 0) for member in team["teamMembers"]]
            members_count = len(member_idxs)
            if members_count == 0:
                continue
            X = sparse.lil_matrix((members_count, X_train.shape[1]), dtype=int)
            X[range(len(member_idxs)), member_idxs] = 1
            
            fail_probas = logreg.predict_proba(X)[:, 0]
            team_proba = 1 - fail_probas.prod() 
            predicted_tournament_results.append(team_proba)

        pred_results[tour_id] = np.array(predicted_tournament_results)
        
    return pred_results

In [72]:
def get_true_results(tournament_ids, results_data):
    results = {}
    for tour_id in tournament_ids:
        tournament = results_data.get(tour_id)
        if not tournament:
            continue 

        result = []
        for team in tournament:
            team['team']['id']
            result.append([team['team']['id'], team.get('questionsTotal', 0)])
        results[tour_id] = np.array(result)
        
    return results

def get_true_results_old(tournament_ids, results_data):
    results = {}
    for tour_id in tournament_ids:
        tournament = results_data.get(tour_id)
        if not tournament:
            continue 
        predicted_tournament_results = []
        for team in tournament:
            answers_mask = team.get("mask")
            if answers_mask:
                ans = len(answers_mask.replace('0', '')) / len(answers_mask)
            else:
                ans = 0
            predicted_tournament_results.append([team['team']['id'], ans])

        results[tour_id] = np.array(predicted_tournament_results)
        
    return results

In [73]:
from scipy.stats import spearmanr, kendalltau
import math

def get_correlations(pred_results, true_results):
    scorrs = []
    kcorrs = []
    for tour_id in true_results:
        #print(true_results[tour_id][:, 1], pred_results[tour_id][:, 1])
        
        scorr = spearmanr(true_results[tour_id][:, 1], pred_results[tour_id][:, 1])[0]
        if not math.isnan(scorr):
            scorrs.append(scorr)

        kcorr = kendalltau(true_results[tour_id][:, 1], pred_results[tour_id][:, 1])[0]
        if not math.isnan(kcorr):
            kcorrs.append(kcorr)
    
    return np.mean(scorrs), np.mean(kcorrs)

In [74]:
# get player_id - weight dict:
player_id_to_weight = get_pid_to_weight(logreg, encoder)

# get predictions and true results
pred_results = get_predictions(test_tournament_ids, results_raw_data, player_id_to_weight)
true_results = get_true_results(test_tournament_ids, results_raw_data)

# get correlations
spearmann_corr, kendall_corr = get_correlations(pred_results, true_results)
print("Spearman's correlation: {}".format(spearmann_corr))
print("Kendall's correlation: {}".format(kendall_corr))

Spearman's correlation: 0.693563569025957
Kendall's correlation: 0.5927910348557992


# 4. EM алгоритм
Теперь главное: ЧГК — это всё-таки командная игра. Поэтому:
- предложите способ учитывать то, что на вопрос отвечают сразу несколько игроков; скорее всего, понадобятся скрытые переменные; не стесняйтесь делать упрощающие предположения, но теперь переменные “игрок X ответил на вопрос Y” при условии данных должны стать зависимыми для игроков одной и той же команды;
- разработайте EM-схему для обучения этой модели, реализуйте её в коде;
- обучите несколько итераций, убедитесь, что целевые метрики со временем растут (скорее всего, ненамного, но расти должны), выберите лучшую модель, используя целевые метрики.


In [94]:
def get_pid_to_weight(model, encoder):
    model_coeffs = model.coef_
    if len(model.coef_.shape) == 2:
        model_coeffs = model.coef_[0]
    
    return {
        encoder.categories_[0][coef_idx]: weight
        for coef_idx, weight in enumerate(model_coeffs[:num_players])
    }

def E_step(X_train, preds):
    tmp_df = X_train.copy()
    tmp_df['probs_reverse'] = 1 - preds
    tmp_df['group_product'] = tmp_df.groupby(['team_id', 'question_id'])['probs_reverse'].transform('prod')
    tmp_df['E_z'] = (preds / (1 - tmp_df['group_product'])) * tmp_df['is_correct']
    return tmp_df['E_z'].values

def M_step(X_train_onehot, z_train):
    linreg = LinearRegression().fit(X_train_onehot, z_train)
    return linreg

In [82]:
num_iter = 5

em_logreg = LogisticRegression(penalty='l2', solver='liblinear').fit(X_train, y_train)
train_preds = logreg.predict_proba(X_train)[:, 1]

player_id_to_weight = get_pid_to_weight(em_logreg, encoder) #gen_pid_weight_dict(logreg, unique_pids, em_logreg)
pred_results =  get_predictions(test_tournament_ids, results_raw_data, player_id_to_weight)# generate_predictions(test_tours_ids, results, player_id_to_weight)
corrs = get_correlations(pred_results, true_results) #calculate_correlations(pred_results, true_results)

spearmann_corr, kendall_corr = get_correlations(pred_results, true_results)

print("Before em starting: ")
print("Spearman's correlation: {}".format(spearmann_corr))
print("Kendall's correlation: {}".format(kendall_corr))

Before em starting: 
Spearman's correlation: 0.693563569025957
Kendall's correlation: 0.5927910348557992


In [96]:
best_model = None
max_scorr = 0

def sig(x):
    return 1 / (1 + np.exp(-x))

def inv_sig(y):
    y = np.clip(y, 1e-8, 1 - 1e-8)
    return np.log(y / (1 - y))

for i in range(num_iter):
    E_z = E_step(train_df, train_preds)
    em_linreg = M_step(X_train, inv_sig(E_z))
    
    player_id_to_weight = get_pid_to_weight(em_linreg, encoder)
    train_preds = sig(em_linreg.predict(X_train))
    pred_results = get_predictions(test_tournament_ids, results_raw_data, player_id_to_weight)
    
    spearmann_corr, kendall_corr  = get_correlations(pred_results, true_results)
    print("=" * 20) 
    print("Iteration {}:".format(i))
    print("Spearman's correlation: {}".format(spearmann_corr))
    print("Kendall's correlation: {}".format(kendall_corr))
    
    if spearmann_corr > max_scorr:
        max_scorr = spearmann_corr
        best_model = em_logreg

C:\Users\v.alehin\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2691: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\Users\v.alehin\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2692: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


Iteration 0:
Spearman's correlation: 0.7640035096516172
Kendall's correlation: 0.637349166818901


C:\Users\v.alehin\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2691: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\Users\v.alehin\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2692: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


Iteration 1:
Spearman's correlation: 0.7640725393388808
Kendall's correlation: 0.6376219266145259


C:\Users\v.alehin\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2691: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\Users\v.alehin\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2692: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


Iteration 2:
Spearman's correlation: 0.7634000470225603
Kendall's correlation: 0.6368865933540319


C:\Users\v.alehin\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2691: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\Users\v.alehin\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2692: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


Iteration 3:
Spearman's correlation: 0.7639034384197392
Kendall's correlation: 0.6374804722227879


C:\Users\v.alehin\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2691: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\Users\v.alehin\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2692: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


Iteration 4:
Spearman's correlation: 0.7635918080477849
Kendall's correlation: 0.6375075497665098


Работает очень долго. Однако существенных улучшений не наблюдается при запуске на более чем 5-10 итераций. Cамое сильное улучшение происходит на 1-4й итерации.

# 5. “Рейтинг-лист” турниров
А что там с вопросами? Постройте “рейтинг-лист” турниров по сложности вопросов. Соответствует ли он интуиции (например, на чемпионате мира в целом должны быть сложные вопросы, а на турнирах для школьников — простые)? Если будет интересно: постройте топ сложных и простых вопросов со ссылками на конкретные записи в базе вопросов ЧГК (это чисто техническое дело, тут никакого ML нету).

In [146]:
question_id_to_weight = {
    encoder.categories_[0][coef_idx]: weight
    for coef_idx, weight in enumerate(best_model.coef_[0][num_players:])
}

tour_rating = []
cur_question_id = 15

for key in tournaments_raw_data.keys():   
    
    questions_arr = tournament_to_question.get(key)
    if questions_arr is None:
        continue

    tournament_weight = 0
    questions_num = 0
    for question in questions_arr:
        if question in question_id_to_weight:
            tournament_weight += question_id_to_weight[question]
            questions_num += 1
    if questions_num != 0:
        tournament_weight /= questions_num
        tour_rating.append((key, tournament_weight))


tour_rating = sorted(tour_rating, key=lambda x: x[1])

Наиболее сложные турниры:

In [147]:
for i, (tour_id, _) in enumerate(tour_rating[:20]):
    name = tournaments_raw_data[tour_id]['name']
    print(f'{i + 1}. {name}')

1. Geek-ЧГК
2. Синхрон-lite. Выпуск XXXI
3. ОВСЧ. 6 этап
4. Кубок Тель-Авива
5. Онлайн: 08:00 Лига старта-13
6. Синхрон Первенства Сибири
7. Научпоп
8. Гран-при Славянки. 2 этап
9. ТРИОтлон-4
10. Международный Карагандинский Синхрон
11. Золотые огни: часть 1
12. Славянка без раздаток. 2 этап
13. Межфакультетский кубок МГУ. Отбор №3
14. Школьный Синхрон-lite. Выпуск 3.2
15. Линч
16. Vilnius Open
17. Чемпионат Минска. Лига Б. Тур пятый
18. Чемпионат Туркменистана
19. Кубок губернатора Иркутской области
20. Кубок Бесконечности. V этап


Наиболее прострые турниры:

In [148]:
for i, (tour_id, _) in enumerate(tour_rating[-20:]):
    name = tournaments_raw_data[tour_id]['name']
    print(f'{i + 1}. {name}')

1. Лига Сибири. IV тур.
2. Школьнае люстэрка
3. Нестерка
4. Умами
5. Январское диминуэндо
6. (а)Синхрон-lite. Лига старта. Эпизод XXL
7. Осенняя поляна
8. Синхрон Кеста
9. Чемпионат Самары
10. Студенческий чемпионат Москвы
11. Мемориал Дмитрия Коноваленко
12. Коломенский Кремль
13. Кубок Золотой Свиньи
14. Скважина Знаний
15. Игра Первопрестольной. Четвёртый сезон
16. Мемориал Дмитрия Павловца
17. Кубок главы города Иваново
18. Первенство правого полушария
19. Антибинго
20. Львов зимой. Барон


Какой-то явной закономерности в том, какие турниры попали в список наиболее сложных или наиболее простых - не наблюдается. Ожидал, что в наиболее сложные попадут "Чемпионат Мира", а в наиболее простые - "Школьные турниры". Однако этого не произошло - таковых турниров бувально парочка. Скорее всего что-то не учтено в модели, что грустно. 